In [1]:
import os
import re
import pickle
import numpy as np
from pprint import pprint
from lxml import etree
from tqdm.notebook import tqdm

In [2]:
input_path = os.path.abspath("../acs-data/unzip-files/")
output_path = os.path.abspath("../acs-data/processed-files/")

In [3]:
# function to collect matching files and dirs
def collect_files(root, res, pattern="", collect_dirs=True):
    
    # go through all item in the dir
    for item in os.listdir(root):
        
        # process item
        item_path = os.path.join(root, item)
        item_is_dir = os.path.isdir(item_path)
        
        # pull valid file in res
        if re.match(pattern, item_path):
            if not item_is_dir or collect_dirs:
                res.append(item_path)
        
        # recursively collect all files
        if item_is_dir:
            collect_files(item_path, res, pattern, collect_dirs)

In [4]:
# collect all xml files
xml_paths = []
collect_files(input_path, xml_paths, pattern=".*\.xml$", collect_dirs=False)
print(f"total xml files: %d" % len(xml_paths))

total xml files: 1545


In [5]:
# helps to extract text from paragraph
def p_helper(node):
    
    # <p/> does not have text
    if node.text is None:
        return ""
    
    # each paragarph is put into a line
    line_list = [node.text]
    for child in node:

        # get the text inside the child if the tag isn't 
        # named-content and inline-formula
        # and the text following the child
        if not child.tag in ("named-content", "inline-formula"):
            line_list.append(" ".join(child.xpath(".//text()")))
        line_list.append(child.tail)

    # there might be none in line_list
        
    # re dark magic
    line = " ".join(line_list)
    line = line.strip()
    line = line.replace("\n", " ")

    # clean up consecutive spaces
    line = re.sub("\s+", " ", line)

    # fix the space around punctuation
    line = re.sub("\s([.,\):;])", r"\1", line)
    line = re.sub("\(\s", r"(", line)
    line = re.sub("\s*([-/])\s*", r"\1", line)
    return line

In [6]:
def kwd_helper(node):
    
    # return a keyword string
    kwd_tokens = node.xpath(".//text()")
    kwd = " ".join(kwd_tokens).replace("\n", " ").strip()
    kwd = re.sub("\s+", " ", kwd)
    return kwd

In [7]:
# this returns interesting titles
# for example: intro, method, and results
# return None for non interesting titles
def title_helper(node):
    
    # extract text from title node
    title = " ".join(node.xpath(".//text()"))
    title = title.replace("\n", " ")
    title = re.sub("\s+", " ", title)
    title = title.strip()
    title = title.lower()
    
    print(title)
    
    # categorize title
    res = []
    if "intro" in title:
        res.append("introduction")
    if "result" in title:
        res.append("result")
    if "discuss" in title:
        res.append("discussion")
    if "material" in title:
        res.append("materials")
    if "method" in title or "procedure" in title:
        res.append("method")
    if "summary" in title:
        res.append("summary")
    return res

In [8]:
def extract_body(root):
    
    # we are interested in the text in the body section
    curr_title = []
    text = []
    text_nodes = root.xpath("/article/body//*[self::p or (self::title and not(ancestor::caption))]")
    for text_node in text_nodes:
        
        # handle title
        if text_node.tag == "title":
            tmp_title = title_helper(text_node)
            if (tmp_title):
                curr_title = tmp_title
        
        # handle paragraph
        elif text_node.tag == "p":
            text.append({
                "text": p_helper(text_node),
                "section": curr_title
            })
    return text

In [9]:
def extract_abstract(root):
    
    # get the abstract paragraph
    abstract = []
    abstract_nodes = root.xpath("//abstract/p")
    if abstract_nodes:
        abstract.append(p_helper(abstract_nodes[0]))
    return abstract

In [10]:
def extract_keywords(root):
    
    # get the keywords
    keywords = []
    kwd_nodes = root.xpath("//kwd-group/kwd")
    for kwd_node in kwd_nodes:
        keywords.append(kwd_helper(kwd_node))
    return keywords

In [11]:
# parse the files
for xml_path in tqdm(xml_paths):
    
    print("\nparsing %s" % xml_path)
    
    # get the root of the xml
    root = etree.parse(xml_path).getroot()
    
    # create a dictionary holding the xml data
    xml_data = {
        "keywords": extract_keywords(root),
        "abstract": extract_abstract(root),
        "body": extract_body(root)
    }
    
    # pickle the data
    # name the file to <pub #>.pkl
    pub_num = xml_path.split("/")[-1].split(".")[0]
    with open(os.path.join(output_path, pub_num + ".pkl"), "wb") as f:
        pickle.dump(xml_data, f)


parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb7b00050/sb7b00050.xml
results
purification and characterization of acecas9 ribonucleoprotein particle
acecas9 has a double cytosine pam
dna unwinding limits acecas9 cleavage efficiency
acecas9 cleaves negatively supercoiled dna efficiently
acecas9 has low off-target cleavage activity for relaxed dna
acecas9 cleaves dna efficiently in escherichia coli
discussion
experimental procedures
cloning of acecas9 and acecas9 mutant
protein expression and purification
in vitro transcription and purification of sgrna
pam library construction
oligonucleotide dna cleavage assay
plasmid dna cleavage assay
generation of negatively supercoiled dna by dna gyrase
single-turnover kinetic assay
bacteria-based positive selection assay

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb6b00301/sb6b00301.xml
introduction
results and discussion
analysis of existing rna thermometers
computational analysis of a thermometer library suggest

methods
bacterial strains, plasmids, and growth conditions
identification of hmo and mdlb orthologs
construction of the s. coelicolor l -hpg pathway mutant
cloning of the orthologous genes into pij10257 and pet28­(+)
complementation of s. coelicolor lw107 with hmo and mdlb orthologous genes
purification of the n-terminally his-tagged proteins
western blot analysis
enzyme activity determinations
uhplc-ms analysis of cda production

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00493/sb8b00493.xml
results and discussion
methods
cell-free experiment preparation and execution
pcr product preparation and plasmid dna assembly
in vivo experiment
microfluidic device experiment

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00218/sb8b00218.xml
results and discussion
optimizing s. oneidensis control over nanoparticle biosynthesis and cellular viability
loss of outer membrane cytochromes decreases pd nanoparticle size and alters cellular localization
loss of ou

hif-2α knockdown
cell viability assays
statistical analysis

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00429/sb8b00429.xml
results
initial design of crispri-mediated repression system for isopentenol production
optimization of a tunable crispri system
crispri negatively affects gene expression in the same operon
efficient multiplex repression of crispri system reduced acetate production
redirecting metabolic flux for isopentenol production
discussion
conclusion
materials and methods
construction of plasmids
strains, medium, and growth
analysis and detection of isopentenol
analysis and detection of rfp and gfp fluorescence

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb300044r/sb300044r.xml
motif control of input/output dynamics
an incoherent feedforward loop can generate a biphasic response
bistability arises via positive feedback
increasing complexity in motif design can lead to autonomous dynamics
cellular adaptation occurs via negative feedback 

split 2 -cytokine protein-ligation: combination of split-il-6 variants and split-inteins generates i-h-il-6 ab‑cd -fc
combination of split-il-11 variants and split-inteins generates h-il-11
methods
cells and reagents
cloning
structural analysis
western blotting
cytokine stimulation of cells for subsequent stat3 analysis
enzyme-linked-immuno-sorbent-assay (elisa)
proliferation assays
affinity chromatography

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb7b00362/sb7b00362.xml
results and discussion
methods
strains and growth conditions
crispr plasmid construction
n. oceanica transformation
nanoluciferase luminescence assays
n. oceanica colony pcr
episomal dna isolation from n. oceanica
episome rescue
episome curing
segregation efficiency
immunoblotting
confocal microscopy
southern-blot analysis

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb6b00201/sb6b00201.xml
results and discussion
methods
bacterial strains and growth conditions
plasmid construction
mea

cell culture
construction of targeting vectors
mmct
transfection of dt40 and cho cells
genomic pcr analysis
fish analysis
mfish analysis
generation of chimeric mice
fcm analyses of mouse es cells
fcm analyses for tissues and blood cells
insertion of multiple integration sites into mac2

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb400069w/sb400069w.xml
introduction
results and discussion
sensing potential as a reliable predictor of temporal qs dynamics
experimental methodology to quantify qs-mediated activation
materials and methods
mathematical modeling of diverse qs systems
strains, media, and growth conditions
processing plate reader data

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb5b00045/sb5b00045.xml
andrew e. blanchard
mihails delmans
bin jia
hanson lee
jiazhang lian
joanna lipinski-kruszka
juergen mairhofer
junwen mao
ryo mizuuchi
morten nørholm
tong si
gerald striedner
maulik thaker
nanxi wang
weston whitaker
oliver wright

parsing /home/jia

...

results and discussion
in vitro packaging mediated approach
targeted cloning of natural product pathways
in vitro editing for heterologous expression of pathways
conclusion
materials and methods
gdna isolation
preparation of cas9 protein and sgrnas
protocols of in vitro packaging mediated approach
in vitro crispr/cas9-mediated editing of stu , heterologous expression, and detection

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb5b00138/sb5b00138.xml
materials and methods
chemicals
molecular phylogenetic analysis by maximum likelihood
expression and purification of sats
enzymatic activity and the kinetic parameter assays
stereopurity determination of β-valienamine by hplc with precolumn derivatization using opa
β-valienamine purification from the reaction mixture and the fermentation broth of δ valc :: btrr
high resolution-mass spectrometry (hr-ms) and nuclear magnetic resonance spectrometry (nmr)
construction of δ valc :: btrr mutant
fermentation of s. hygroscopicus 500


parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00086/sb8b00086.xml
methods
construction of plasmids and transformation
media and cultivations
individual protein expression and purification
enzymatic activity assay
metabolites and glucose quantification
dynamic light scattering assay
scanning electron, field-emission scanning electron and atomic force microscopy
fluorescence complementation assay
crispr-cas9 mediated genome editing

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb400102w/sb400102w.xml
results and discussion
methods
enzymes, chemicals, and strain
plasmid construction
assay protocol
fluorescence measurements
numerics
determination of the region of the single stable stationary point achieved by gene overexpression

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb300130z/sb300130z.xml
yidan ding
joshua t. kittleson
dante w. romanini

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00223/sb8b00223.xml
results
desi

chromosomal integration of ackrs enhances regulated amber suppression
four broad-host-range cloning vectors allow amber suppression in diverse bacterial species
concluding remarks
materials and methods
bacterial stains, oligonucleotides and plasmids
molecular biological methods
plasmid and strain construction
lux-based luminescence activity assay
nanoluc luciferase assay in s. enterica and v. cholerae

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb5b00261/sb5b00261.xml
methods
plasmid construction
ai-2 assay
protein expression assays
rna isolation and analysis

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb7b00115/sb7b00115.xml
results and discussion
conclusion
methods
synthesis of unnatural triphosphates
steady-state kinetics assay

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00138/sb8b00138.xml

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00487/sb8b00487.xml
results and discussion
materials and methods
media, pla

materials and methods
reporter strain construction
yfp reporter assays
vanillin-β-glucoside production strains
bioreactor cultures and hplc analysis of metabolites
rnaseq and transcriptome data analysis
database for rna-seq data

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb400037a/sb400037a.xml
methods
library and plasmid construction
cell culture and transfection
egfp assay
seap assay

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb9b00187/sb9b00187.xml
materials and methods
chemicals and reagents
cloning of plasmids
recombinant protein expression and purification
protein purification
isopeptide bond formation between spytag-adh and eutm-spycatcher tested by sds-page
negative stain transmission electron microscopy
activity and stability of (spytag)-adh immobilized on scaffolds
results and discussion
design and in vitro characterization of a toolbox of protein scaffolds
rapid covalent enzyme immobilization on eutm-spycatcher scaffolds
eutm-spycatcher sc


parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb3000639/sb3000639.xml
methods

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb8b00102/sb8b00102.xml
results and discussion
construction and characterization of a promoter library for e. coli
characterization of the promoter library in different bacteria
application of the promoter library to synthesize pha in h. bluephagenesis
pha production by recombinant h. bluephagenesis with optimized promoter
performance of h. bluephagenesis with optimized promoter in a 7 l fermentor
conclusions
materials and methods
bacterial strains, media and chemicals
plasmid library construction and conjugation
flow cytometry and characterization of promoter strengths
genome integration
shake flask experiment
fermentor studies
pha analytical methods

parsing /home/jiawei/sbks-ucsd/biobert/acs-data/unzip-files/sb7b00078/sb7b00078.xml
methods
peptide design and preparation
screening of synthetic α-factor analogues
construction of expr

In [12]:
# load some file to check
pkl_paths = []
collect_files(output_path, pkl_paths, pattern=".*\.pkl$", collect_dirs=False)
print("pkl files found: %d" % len(pkl_paths))

pkl files found: 1545


In [13]:
with open(pkl_paths[101], "rb") as f:
    pprint(pickle.load(f))

{'abstract': ['Metabolic engineering and synthetic biology usually require '
              'universal expression systems for stable and efficient gene '
              'expression in various organisms. In this study, a '
              'host-independent and stable T7 expression system had been '
              'developed by integrating T7 RNA polymerase and its cognate '
              'transcriptional units in single plasmid. The expression of T7 '
              'RNA polymerase was restricted below its lethal threshold using '
              'a T7 RNA polymerase antisense gene cassette, which allowed long '
              'periods of cultivation and protein production. In addition, by '
              'designing ribosome binding sites, we further tuned the '
              'expression capacity of this novel T7 system within a wide '
              'range. This host-independent expression system efficiently '
              'expressed genes in five different Gram-negative strains and one '
     